In [3]:
!pip install pyspark

Processing /Users/aleksandrsavchenko/Library/Caches/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471/pyspark-2.4.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e3/53/c737818eb9a7dc32a7cd4f1396e787bd94200c3997c72c1dbe028587bd76/py4j-0.10.7-py2.py3-none-any.whl


In [13]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit, avg
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import QuantileDiscretizer

In [14]:
spark = SparkSession.builder.appName("PySparkTitanikJob").getOrCreate()

In [15]:
spark

In [16]:
titanic_df = spark.read.csv('train.csv', header = 'True', inferSchema='True')

In [17]:
display(titanic_df)

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]

In [18]:
titanic_df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [19]:
passengers_count = titanic_df.count()

In [20]:
titanic_df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [21]:
gropuBy_output = titanic_df.groupBy("Survived").count()

In [22]:
gropuBy_output.show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



In [23]:
avg_age = round(titanic_df.select(avg(col('Age'))).collect()[0][0],0)

In [24]:
avg_age

30.0

In [25]:
titanic_df = titanic_df.fillna({'Age': avg_age})

In [26]:
titanic_df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|30.0|    0|    0|      

In [27]:
titanic_df = titanic_df.fillna({"Embarked" : 'S'})

In [28]:
titanic_df = titanic_df.drop("Cabin")

In [29]:
titanic_df = titanic_df.withColumn("Family_Size",col('SibSp')+col('Parch'))

In [30]:
titanic_df = titanic_df.withColumn('Alone',lit(0))

In [31]:
titanic_df = titanic_df.withColumn("Alone",when(titanic_df["Family_Size"] == 0, 1).otherwise(titanic_df["Alone"]))

In [32]:
data_df = titanic_df

In [33]:
titanic_df.show(2)

+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+--------+-----------+-----+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|   Ticket|   Fare|Embarked|Family_Size|Alone|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+--------+-----------+-----+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|A/5 21171|   7.25|       S|          1|    0|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0| PC 17599|71.2833|       C|          1|    0|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+--------+-----------+-----+
only showing top 2 rows



In [34]:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(titanic_df) for column in ["Sex","Embarked"]]
pipeline = Pipeline(stages=indexers)
titanic_df = pipeline.fit(titanic_df).transform(titanic_df)

In [35]:
titanic_df = titanic_df.drop("PassengerId","Name","Ticket","Cabin","Embarked","Sex")

In [36]:
titanic_df.show()

+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+
|Survived|Pclass| Age|SibSp|Parch|   Fare|Family_Size|Alone|Sex_index|Embarked_index|
+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+
|       0|     3|22.0|    1|    0|   7.25|          1|    0|      0.0|           0.0|
|       1|     1|38.0|    1|    0|71.2833|          1|    0|      1.0|           1.0|
|       1|     3|26.0|    0|    0|  7.925|          0|    1|      1.0|           0.0|
|       1|     1|35.0|    1|    0|   53.1|          1|    0|      1.0|           0.0|
|       0|     3|35.0|    0|    0|   8.05|          0|    1|      0.0|           0.0|
|       0|     3|30.0|    0|    0| 8.4583|          0|    1|      0.0|           2.0|
|       0|     1|54.0|    0|    0|51.8625|          0|    1|      0.0|           0.0|
|       0|     3| 2.0|    3|    1| 21.075|          4|    0|      0.0|           0.0|
|       1|     3|27.0|    0|    2|11.1333|          2|

In [37]:
feature = VectorAssembler(inputCols=titanic_df.columns[1:],outputCol="features")
feature_vector= feature.transform(titanic_df)

In [38]:
feature_vector.show()

+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+--------------------+
|Survived|Pclass| Age|SibSp|Parch|   Fare|Family_Size|Alone|Sex_index|Embarked_index|            features|
+--------+------+----+-----+-----+-------+-----------+-----+---------+--------------+--------------------+
|       0|     3|22.0|    1|    0|   7.25|          1|    0|      0.0|           0.0|[3.0,22.0,1.0,0.0...|
|       1|     1|38.0|    1|    0|71.2833|          1|    0|      1.0|           1.0|[1.0,38.0,1.0,0.0...|
|       1|     3|26.0|    0|    0|  7.925|          0|    1|      1.0|           0.0|[3.0,26.0,0.0,0.0...|
|       1|     1|35.0|    1|    0|   53.1|          1|    0|      1.0|           0.0|[1.0,35.0,1.0,0.0...|
|       0|     3|35.0|    0|    0|   8.05|          0|    1|      0.0|           0.0|(9,[0,1,4,6],[3.0...|
|       0|     3|30.0|    0|    0| 8.4583|          0|    1|      0.0|           2.0|[3.0,30.0,0.0,0.0...|
|       0|     1|54.0|    0|    0|51.

In [39]:
(training_data, test_data) = feature_vector.randomSplit([0.8, 0.2],seed = 42)

In [40]:
training_data.show()

+--------+------+----+-----+-----+--------+-----------+-----+---------+--------------+--------------------+
|Survived|Pclass| Age|SibSp|Parch|    Fare|Family_Size|Alone|Sex_index|Embarked_index|            features|
+--------+------+----+-----+-----+--------+-----------+-----+---------+--------------+--------------------+
|       0|     1| 2.0|    1|    2|  151.55|          3|    0|      1.0|           0.0|[1.0,2.0,1.0,2.0,...|
|       0|     1|21.0|    0|    1| 77.2875|          1|    0|      0.0|           0.0|[1.0,21.0,0.0,1.0...|
|       0|     1|22.0|    0|    0|135.6333|          0|    1|      0.0|           1.0|[1.0,22.0,0.0,0.0...|
|       0|     1|24.0|    0|    0|    79.2|          0|    1|      0.0|           1.0|[1.0,24.0,0.0,0.0...|
|       0|     1|24.0|    0|    1|247.5208|          1|    0|      0.0|           1.0|[1.0,24.0,0.0,1.0...|
|       0|     1|25.0|    1|    2|  151.55|          3|    0|      1.0|           0.0|[1.0,25.0,1.0,2.0...|
|       0|     1|27.0|    0|

# ML models

# LogisticRegression

In [41]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="Survived", featuresCol="features")
#Training algo
lrModel = lr.fit(training_data)
lr_prediction = lrModel.transform(test_data)
lr_prediction.select("prediction", "Survived", "features").show(5)
evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")

+----------+--------+--------------------+
|prediction|Survived|            features|
+----------+--------+--------------------+
|       1.0|       0|[1.0,18.0,1.0,0.0...|
|       1.0|       0|[1.0,19.0,1.0,0.0...|
|       0.0|       0|[1.0,19.0,3.0,2.0...|
|       1.0|       0|[1.0,29.0,1.0,0.0...|
|       0.0|       0|(9,[0,1,4,6],[1.0...|
+----------+--------+--------------------+
only showing top 5 rows



In [42]:
lr_accuracy = evaluator.evaluate(lr_prediction)
print("LogisticRegression [Accuracy] = %g"% (lr_accuracy))
print("LogisticRegression [Error] = %g " % (1.0 - lr_accuracy))

LogisticRegression [Accuracy] = 0.793103
LogisticRegression [Error] = 0.206897 


# DecisionTreeClassifier

In [43]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(labelCol="Survived", featuresCol="features")
dt_model = dt.fit(training_data)
dt_prediction = dt_model.transform(test_data)

dt_prediction.select("prediction", "Survived", "features").show(5)

+----------+--------+--------------------+
|prediction|Survived|            features|
+----------+--------+--------------------+
|       0.0|       0|[1.0,18.0,1.0,0.0...|
|       0.0|       0|[1.0,19.0,1.0,0.0...|
|       0.0|       0|[1.0,19.0,3.0,2.0...|
|       0.0|       0|[1.0,29.0,1.0,0.0...|
|       0.0|       0|(9,[0,1,4,6],[1.0...|
+----------+--------+--------------------+
only showing top 5 rows



In [44]:
dt_accuracy = evaluator.evaluate(dt_prediction)
print("DecisionTreeClassifier [Accuracy] = %g"% (dt_accuracy))
print("DecisionTreeClassifier [Error] = %g " % (1.0 - dt_accuracy))

DecisionTreeClassifier [Accuracy] = 0.798851
DecisionTreeClassifier [Error] = 0.201149 


# RandomForestClassifier

In [45]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="Survived", featuresCol="features")
rf_model = rf.fit(training_data)
rf_prediction = rf_model.transform(test_data)
rf_prediction.select("prediction", "Survived", "features").show(5)

+----------+--------+--------------------+
|prediction|Survived|            features|
+----------+--------+--------------------+
|       1.0|       0|[1.0,18.0,1.0,0.0...|
|       0.0|       0|[1.0,19.0,1.0,0.0...|
|       1.0|       0|[1.0,19.0,3.0,2.0...|
|       0.0|       0|[1.0,29.0,1.0,0.0...|
|       0.0|       0|(9,[0,1,4,6],[1.0...|
+----------+--------+--------------------+
only showing top 5 rows



In [46]:
rf_accuracy = evaluator.evaluate(rf_prediction)
print("RandomForestClassifier [Accuracy] = %g"% (rf_accuracy))
print("RandomForestClassifier [Error] = %g" % (1.0 - rf_accuracy))

RandomForestClassifier [Accuracy] = 0.804598
RandomForestClassifier [Error] = 0.195402


# Gradient-boosted tree classifier

In [47]:
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(labelCol="Survived", featuresCol="features",maxIter=10)
gbt_model = gbt.fit(training_data)
gbt_prediction = gbt_model.transform(test_data)
gbt_prediction.select("prediction", "Survived", "features").show(5)

+----------+--------+--------------------+
|prediction|Survived|            features|
+----------+--------+--------------------+
|       0.0|       0|[1.0,18.0,1.0,0.0...|
|       1.0|       0|[1.0,19.0,1.0,0.0...|
|       0.0|       0|[1.0,19.0,3.0,2.0...|
|       1.0|       0|[1.0,29.0,1.0,0.0...|
|       0.0|       0|(9,[0,1,4,6],[1.0...|
+----------+--------+--------------------+
only showing top 5 rows



In [48]:
gbt_accuracy = evaluator.evaluate(gbt_prediction)
print("Gradient-boosted [Accuracy] = %g"% (gbt_accuracy))
print("Gradient-boosted [Error] = %g"% (1.0 - gbt_accuracy))

Gradient-boosted [Accuracy] = 0.793103
Gradient-boosted [Error] = 0.206897


# Save & Load Model

In [49]:
rf_model.save('rf_model')

In [50]:
from pyspark.ml.classification import RandomForestClassificationModel
type(RandomForestClassificationModel.load('rf_model'))

pyspark.ml.classification.RandomForestClassificationModel

# Pipeline

In [66]:
from pyspark.ml.pipeline import PipelineModel

In [90]:
data_df = data_df.drop("PassengerId","Name","Ticket","Cabin")

In [120]:
train, validate = data_df.randomSplit([0.8, 0.2])

In [121]:
train.show(5)

+--------+------+------+----+-----+-----+------+--------+-----------+-----+
|Survived|Pclass|   Sex| Age|SibSp|Parch|  Fare|Embarked|Family_Size|Alone|
+--------+------+------+----+-----+-----+------+--------+-----------+-----+
|       0|     1|female| 2.0|    1|    2|151.55|       S|          3|    0|
|       0|     1|female|25.0|    1|    2|151.55|       S|          3|    0|
|       0|     1|  male|18.0|    1|    0| 108.9|       C|          1|    0|
|       0|     1|  male|19.0|    1|    0|  53.1|       S|          1|    0|
|       0|     1|  male|19.0|    3|    2| 263.0|       S|          5|    0|
+--------+------+------+----+-----+-----+------+--------+-----------+-----+
only showing top 5 rows



In [122]:
indexer_sex = StringIndexer(inputCol="Sex", outputCol="Sex_index")

In [123]:
indexer_embarked = StringIndexer(inputCol="Embarked", outputCol="Embarked_index")

In [124]:
feature = VectorAssembler(
    inputCols=["Pclass","Age","SibSp","Parch","Fare","Family_Size","Embarked_index","Sex_index"],
    outputCol="features")


In [125]:
rf_classifier = RandomForestClassifier(labelCol="Survived", featuresCol="features")

In [126]:
pipeline = Pipeline(stages=[indexer_sex, indexer_embarked, feature, rf_classifier])

In [127]:
p_model = pipeline.fit(train)

In [128]:
type(p_model)

pyspark.ml.pipeline.PipelineModel

In [129]:
p_model.write().overwrite().save('p_model')

In [130]:
model = PipelineModel.load('p_model')

In [131]:
prediction = p_model.transform(validate)

In [133]:
validate.show(5)

+--------+------+------+----+-----+-----+--------+--------+-----------+-----+
|Survived|Pclass|   Sex| Age|SibSp|Parch|    Fare|Embarked|Family_Size|Alone|
+--------+------+------+----+-----+-----+--------+--------+-----------+-----+
|       0|     1|female|50.0|    0|    0| 28.7125|       C|          0|    1|
|       0|     1|  male|24.0|    0|    0|    79.2|       C|          0|    1|
|       0|     1|  male|24.0|    0|    1|247.5208|       C|          1|    0|
|       0|     1|  male|29.0|    0|    0|    30.0|       S|          0|    1|
|       0|     1|  male|30.0|    0|    0|  25.925|       S|          0|    1|
+--------+------+------+----+-----+-----+--------+--------+-----------+-----+
only showing top 5 rows



In [132]:
prediction.select(["Pclass","Age","SibSp","Parch","Fare","Family_Size","Embarked_index","Sex_index"]).show(5)

+------+----+-----+-----+--------+-----------+--------------+---------+
|Pclass| Age|SibSp|Parch|    Fare|Family_Size|Embarked_index|Sex_index|
+------+----+-----+-----+--------+-----------+--------------+---------+
|     1|50.0|    0|    0| 28.7125|          0|           1.0|      1.0|
|     1|24.0|    0|    0|    79.2|          0|           1.0|      0.0|
|     1|24.0|    0|    1|247.5208|          1|           1.0|      0.0|
|     1|29.0|    0|    0|    30.0|          0|           0.0|      0.0|
|     1|30.0|    0|    0|  25.925|          0|           0.0|      0.0|
+------+----+-----+-----+--------+-----------+--------------+---------+
only showing top 5 rows



In [108]:
prediction.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = false)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = false)
 |-- Family_Size: integer (nullable = true)
 |-- Alone: integer (nullable = false)
 |-- Sex_index: double (nullable = false)
 |-- Embarked_index: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [134]:
evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")

In [135]:
p_accuracy = evaluator.evaluate(prediction)
print("Pipeline model [Accuracy] = %g"% (p_accuracy))
print("Pipeline model [Error] = %g " % (1.0 - p_accuracy))

Pipeline model [Accuracy] = 0.828571
Pipeline model [Error] = 0.171429 
